# Librerías

In [ ]:
from clever_cpmp.utils import load_data
from clever_cpmp.utils import load_data_mongo
from clever_cpmp.utils import connect_to_server
from clever_cpmp.model import load_cpmp_model
from clever_cpmp.model import create_model
from clever_cpmp.model import reinforcement_training
from cpmp_ml.optimizer import GreedyV1
from cpmp_ml.optimizer import GreedyV2
from cpmp_ml.utils.adapters import AttentionModel
from cpmp_ml.validations import validate_model

# Cargar Data

In [ ]:
MONGO_URI_MODEL_V2 = 'mongodb+srv://<Username>:<Password>@cluster0.p9y0etq.mongodb.net/'
MONGO_URI_LOCAL = "mongodb://localhost:27017/"

## Local

In [ ]:
data_5x5, labels_5x5 = load_data('Data_CPMP/Data_Model_v2/data_5x5_Multiple.csv')

In [ ]:
data_7x7, labels_7x7 = load_data('data/data_7x7_Multiple.csv')

## MongoDB

### Modelo v2

In [ ]:
client_v2 = connect_to_server(MONGO_URI_MODEL_V2)

base_de_datos = client_v2.data_Model_v2
data_5x5, labels_5x5 = load_data_mongo(base_de_datos.data_5x5)

client_v2.close()

In [ ]:
client_v2 = connect_to_server(MONGO_URI_MODEL_V2)

base_de_datos = client_v2.data_Model_v2
data_7x7, labels_7x7 = load_data_mongo(base_de_datos.data_7x7)

client_v2.close()

### Local

In [ ]:
client_local = connect_to_server(MONGO_URI_LOCAL)

base_de_datos = client_local['data_model_v2-1']
data_5x5 = load_data_mongo(base_de_datos.data_5x5)

client_local.close()

# Cargar Modelos

In [ ]:
model_Sx7 = load_cpmp_model("./models/model_Sx7.h5")

# Modelo Sx5

In [ ]:
model_Sx5 = create_model(5, 5, 'Adam', num_stacks= 3)

## Entrenamiento

In [ ]:
for stack in data_5x5:
    model_Sx5.fit(data_5x5[stack]["States"], data_5x5[stack]["Labels"], batch_size= 32, epochs= 20)

## Testing

In [ ]:
validate_model(model_Sx5, GreedyV1(), AttentionModel(), 5, 5, 15, size_states= 1000)

In [ ]:
validate_model(model_Sx5, GreedyV2(), AttentionModel(), 5, 5, 15, size_states= 1000, max_steps= 30)

In [ ]:
model_Sx5.save("./models/model_Sx5.h5")

# Reinforcement Learning

In [ ]:
reinforcement_training(model_Sx5, 5, 5, 15, GreedyV2(), AttentionModel(), iter= 2, max_steps= 30, epochs= 10, batch_size= 50)